In [69]:
from sklearn.model_selection import train_test_split
import os
import glob
import re
import numpy as np
import sklearn.metrics.pairwise as mp
import pandas as pd
import scipy
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.neural_network import MLPRegressor
from sklearn.cross_decomposition import PLSRegression



In [70]:
X_train = scipy.sparse.load_npz("data/processed_descriptions/train_descriptions_noLemma.npz")
X_test  = scipy.sparse.load_npz("data/processed_descriptions/test_descriptions_noLemma.npz")

X_train_nouns = scipy.sparse.load_npz("data/processed_descriptions/train_descriptions_nouns_noLemma.npz")
X_test_nouns  = scipy.sparse.load_npz("data/processed_descriptions/test_descriptions_nouns_noLemma.npz")

X_train_verbs = scipy.sparse.load_npz("data/processed_descriptions/train_descriptions_verbs.npz")
X_test_verbs  = scipy.sparse.load_npz("data/processed_descriptions/test_descriptions_verbs.npz")

In [71]:
Y_train = scipy.sparse.load_npz("data/processed_tags/train_tag_nocat_tfdif.npz")
Y_test  = scipy.sparse.load_npz("data/processed_tags/test_tag_nocat_tfdif.npz")

In [72]:
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(10000, 8339) (2000, 8339) (10000, 92) (2000, 92)


In [73]:
resnet_fc1000_train = pd.read_csv("data/features_train/features_resnet1000_train.csv", header = None, index_col = 0)
resnet_train = pd.read_csv("data/features_train/features_resnet1000intermediate_train.csv", header = None, index_col = 0)

resnet_fc1000_test = pd.read_csv("data/features_test/features_resnet1000_test.csv", header = None, index_col = 0)
resnet_test = pd.read_csv("data/features_test/features_resnet1000intermediate_test.csv", header = None, index_col = 0)

print(resnet_fc1000_train.shape, resnet_fc1000_test.shape)
print(resnet_train.shape, resnet_test.shape)

(10000, 1000) (2000, 1000)
(10000, 2048) (2000, 2048)


In [74]:
tr_idx = resnet_train.index
print(tr_idx[0:10])
new_idx1 = [int(re.findall("\d+", idx)[0]) for idx in tr_idx]
print(new_idx1[0:10])
resnet_train = np.array(pd.DataFrame(np.array(resnet_train), index = new_idx1).sort_index())

te_idx = resnet_test.index
print(te_idx[0:10])
new_idx2 = [int(re.findall("\d+", idx)[0]) for idx in te_idx]
print(new_idx2[0:10])
resnet_test = np.array(pd.DataFrame(np.array(resnet_test), index = new_idx2).sort_index())

Index(['images_train/5373.jpg', 'images_train/984.jpg',
       'images_train/7127.jpg', 'images_train/5293.jpg',
       'images_train/3688.jpg', 'images_train/3340.jpg',
       'images_train/4787.jpg', 'images_train/5707.jpg',
       'images_train/1262.jpg', 'images_train/8355.jpg'],
      dtype='object', name=0)
[5373, 984, 7127, 5293, 3688, 3340, 4787, 5707, 1262, 8355]
Index(['images_test/152.jpg', 'images_test/901.jpg', 'images_test/1609.jpg',
       'images_test/501.jpg', 'images_test/517.jpg', 'images_test/1516.jpg',
       'images_test/1822.jpg', 'images_test/1993.jpg', 'images_test/405.jpg',
       'images_test/1718.jpg'],
      dtype='object', name=0)
[152, 901, 1609, 501, 517, 1516, 1822, 1993, 405, 1718]


In [75]:
fc_tr_idx = resnet_fc1000_train.index
print(fc_tr_idx[0:10])
fc_new_idx1 = [int(re.findall("\d+", idx)[0]) for idx in fc_tr_idx]
print(fc_new_idx1[0:10])
resnet_fc1000_train = np.array(pd.DataFrame(np.array(resnet_fc1000_train), index = fc_new_idx1).sort_index())

fc_te_idx = resnet_fc1000_test.index
print(fc_te_idx[0:10])
fc_new_idx2 = [int(re.findall("\d+", idx)[0]) for idx in fc_te_idx]
print(fc_new_idx2[0:10])
resnet_fc1000_test = np.array(pd.DataFrame(np.array(resnet_fc1000_test), index = fc_new_idx2).sort_index())

Index(['images_train/5373.jpg', 'images_train/984.jpg',
       'images_train/7127.jpg', 'images_train/9609.jpg',
       'images_train/5293.jpg', 'images_train/3688.jpg',
       'images_train/3340.jpg', 'images_train/4787.jpg',
       'images_train/5707.jpg', 'images_train/1262.jpg'],
      dtype='object', name=0)
[5373, 984, 7127, 9609, 5293, 3688, 3340, 4787, 5707, 1262]
Index(['images_test/152.jpg', 'images_test/901.jpg', 'images_test/1609.jpg',
       'images_test/501.jpg', 'images_test/517.jpg', 'images_test/1822.jpg',
       'images_test/1993.jpg', 'images_test/832.jpg', 'images_test/1462.jpg',
       'images_test/1718.jpg'],
      dtype='object', name=0)
[152, 901, 1609, 501, 517, 1822, 1993, 832, 1462, 1718]


In [81]:
def map20(test, sim_matrix): 
    total_score = 0
    for num, idx in enumerate(test):
        num =1
        score = 0
        rank = np.argwhere(np.argsort(sim_matrix[num])[::-1] == num)[0][0]
        if rank <20:
            score = (20 + 1-(rank+1))/20
        else:
            score = 0 
        total_score += score
    return total_score

In [82]:
### Cross Validation

test_idx=[]
train_idx=[]


from sklearn.model_selection import train_test_split, KFold
kf = KFold(n_splits=3, random_state=None) 
for tr_idx, te_idx in kf.split(X_train):
    train_idx.append(tr_idx)
    test_idx.append(te_idx)

In [56]:
### PLS forward all 

r2_pls_all_forward = []
r2_pls2_all_reversed = []

for i in range(3):
    print(i)

    pls2_all_forward = PLSRegression(n_components = 100)
    pls2_all_forward.fit(X_train[train_idx[i]].toarray(), resnet_train[train_idx[i]])
    r2_pls_all_forward.append(pls2_all_forward.score(X_train[test_idx[i]].toarray(), resnet_train[test_idx[i]]))
    pls2_all_reversed = PLSRegression(n_components = 100)
    pls2_all_reversed.fit(resnet_train[train_idx[i]], X_train[train_idx[i]].toarray())
    r2_pls2_all_reversed.append(pls2_all_reversed.score(resnet_train[test_idx[i]], X_train[test_idx[i]].toarray()))
    
    print("All to pool5")
    print("Forward: ", r2_pls_all_forward)
    print("Reversed: ", r2_pls2_all_reversed)
    
    
    print('Done with: ', i)
    
    


0


/anaconda3/lib/python3.6/site-packages/sklearn/cross_decomposition/pls_.py:77: UserWarning: Maximum number of iterations reached
  warnings.warn('Maximum number of iterations reached')


All to pool5
Forward:  [0.29163020521408023]
Reversed:  [0.14737533766684474]
Done with:  0
1
All to pool5
Forward:  [0.29163020521408023, 0.30040209294510645]
Reversed:  [0.14737533766684474, 0.14147719909594458]
Done with:  1
2
All to pool5
Forward:  [0.29163020521408023, 0.30040209294510645, 0.2988635868956745]
Reversed:  [0.14737533766684474, 0.14147719909594458, 0.14711494026204033]
Done with:  2


In [83]:
scores = []
map20_scores = []
for i in range(3):
    print(i)
    mlp_all = MLPRegressor(activation = 'relu', 
                         hidden_layer_sizes = (500, 100, 500),
                         max_iter = 200,
                         learning_rate = "adaptive",
                         warm_start = True)
    #print(X_train[train_idx[i]].shape, X_train[test_idx[i]].shape)
    mlp_all.fit(X_train_nouns[train_idx[i]], Y_train[train_idx[i]].toarray())
    pred = mlp_all.predict(X_train_nouns[test_idx[i]])
    sim_matrix = mp.cosine_similarity(pred, Y_train[test_idx[i]])
    map20_scores.append(map20(test=test_idx[i], sim_matrix=sim_matrix)/test_idx[0].shape[0])
    #pred = mlp_pool5.predict(X_test)
    scores.append(mlp_all.score(X_train_nouns[test_idx[i]], Y_train[test_idx[i]].toarray()))
    
    print(scores)
    print(map20_scores)

0
[0.5081152176692415]
[0.5499999999999695]
1
[0.5081152176692415, 0.5263438445091709]
[0.5499999999999695, 0.0]
2
[0.5081152176692415, 0.5263438445091709, 0.5187922765058834]
[0.5499999999999695, 0.0, 0.0]


In [58]:
### PLS forward verbs

r2_pls_verbs_forward = []
#r2_pls2_verbs_reversed = []

for i in range(3):
    print(i)

    pls2_verbs_forward = PLSRegression(n_components = 100)
    pls2_verbs_forward.fit(X_train_verbs[train_idx[i]].toarray(), resnet_train[train_idx[i]])
    r2_pls_verbs_forward.append(pls2_verbs_forward.score(X_train_verbs[test_idx[i]].toarray(), resnet_train[test_idx[i]]))
    #pls2_verbs_reversed = PLSRegression(n_components = 100)
    #pls2_verbs_reversed.fit(resnet_train[train_idx[i]], X_train_verbs[train_idx[i]].toarray())
    #r2_pls2_verbs_reversed.append(pls2_verbs_reversed.score(resnet_train[test_idx[i]]), X_train_verbs[test_idx[i]])
        
    print("Verbs to pool5")
    print("Forward: ", r2_pls_verbs_forward)
    
    
    print('Done with: ', i)
    
    

0


/anaconda3/lib/python3.6/site-packages/sklearn/cross_decomposition/pls_.py:77: UserWarning: Maximum number of iterations reached
  warnings.warn('Maximum number of iterations reached')


Verbs to pool5
Forward:  [0.26223815921371235]
Done with:  0
1
Verbs to pool5
Forward:  [0.26223815921371235, 0.27094558459406004]
Done with:  1
2
Verbs to pool5
Forward:  [0.26223815921371235, 0.27094558459406004, 0.26604520329734316]
Done with:  2


In [78]:
### PLS forward all nouns

r2_pls_nouns_forward = []
r2_pls2_nouns_reversed = []

for i in range(3):
    print(i)
    
    pls2_nouns_forward = PLSRegression(n_components = 100)
    pls2_nouns_forward.fit(X_train_nouns[train_idx[i]].toarray(), resnet_fc1000_train[train_idx[i]])
    r2_pls_nouns_forward.append(pls2_nouns_forward.score(X_train_nouns[test_idx[i]].toarray(), resnet_fc1000_train[test_idx[i]]))
    pls2_nouns_reversed = PLSRegression(n_components = 100)
    pls2_nouns_reversed.fit(resnet_fc1000_train[train_idx[i]], X_train_nouns[train_idx[i]].toarray())
    r2_pls2_nouns_reversed.append(pls2_nouns_reversed.score(resnet_fc1000_train[test_idx[i]], X_train_nouns[test_idx[i]].toarray()))
        
    print("Nouns to fc1000")
    print("Forward: ", r2_pls_nouns_forward)
    print("Reversed: ", r2_pls2_nouns_reversed)
    
    
    print('Done with: ', i)
    
    

0
(6666, 5924) (6666, 1000)


/anaconda3/lib/python3.6/site-packages/sklearn/cross_decomposition/pls_.py:77: UserWarning: Maximum number of iterations reached
  warnings.warn('Maximum number of iterations reached')
/anaconda3/lib/python3.6/site-packages/sklearn/cross_decomposition/pls_.py:49: RuntimeWarning: invalid value encountered in true_divide
  x_weights = np.dot(X.T, y_score) / np.dot(y_score.T, y_score)
/anaconda3/lib/python3.6/site-packages/sklearn/cross_decomposition/pls_.py:285: RuntimeWarning: invalid value encountered in less
  if np.all(np.dot(Yk.T, Yk) < np.finfo(np.double).eps):


KeyboardInterrupt: 